 Building a Equal-weight S&P 500 Index Fund

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL' 
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'   
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 127022758, 'calculationPrice': 'close', 'change': -1.14, 'changePercent': -0.0067, 'close': 176.29, 'closeSource': 'icflofia', 'closeTime': 1688328455139, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 173.86, 'delayedPriceTime': 1708544000490, 'extendedChange': 0.76, 'extendedChangePercent': 0.0045, 'extendedPrice': 178.92, 'extendedPriceTime': 1718671623418, 'high': 177.17, 'highSource': '1ey aileud rpeide mnc5t', 'highTime': 1647032878333, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 173.1, 'iexCloseTime': 1649634434020, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 175.28, 'iexOpenTime': 1671009435585, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1664577909726, 'latestPrice': 171.71, 'latestSource': 'Close', 'latestTime': 'December 17, 2021', 'latestUpdate': 1684530027286, 'latestVolume': 203951124, 'low': 176, 'lowSource': 'e cdl5epeiirtn1 yd

In [5]:
price = data['latestPrice'] 
market_cap = data['marketCap']



In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares'] 
final_dataframe = pd.DataFrame(columns = my_columns) 
final_dataframe 

,Ticker,Stock Price,Market Capitalization,Number of Shares


In [7]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns  
    ),
    ignore_index=True   
)

,Ticker,Stock Price,Market Capitalization,Number of Shares
0,AAPL,171.71,2818157299069,N/A


In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)    
for stock in stocks['Ticker']:  
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'   
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index= True
    )    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []    
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))   
    
final_dataframe = pd.DataFrame(columns = my_columns)    
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'

                ],
                index = my_columns),
                ignore_index=True
        )

final_dataframe

KeyError: 'KSU'